### Kaggle competition Energy consumption prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from myfunc import get_stats
import gc

pd.set_option('display.max_columns', 300)

#### Import necessary data

In [2]:
root = '../data/'

train = pd.read_csv(root + 'train.csv')
weather_train = pd.read_csv(root + 'weather_train.csv')

test = pd.read_csv(root + 'test.csv')
weather_test = pd.read_csv(root + 'weather_test.csv')

building = pd.read_csv(root + 'building_metadata.csv')


In [3]:
print('Size of train data', train.shape)
print('Size of weather_train data', weather_train.shape)
print('Size of weather_test data', weather_test.shape)
print('Size of building data', building.shape)

Size of train data (20216100, 4)
Size of weather_train data (139773, 9)
Size of weather_test data (277243, 9)
Size of building data (1449, 6)


In [4]:
# train.csv
# building_id - Foreign key for the building metadata.
# meter - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, hotwater: 3}. Not every building has all meter types.
# timestamp - When the measurement was taken
# meter_reading - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error.
# building_meta.csv
# site_id - Foreign key for the weather files.
# building_id - Foreign key for training.csv
# primary_use - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
# square_feet - Gross floor area of the building
# year_built - Year building was opened
# floor_count - Number of floors of the building
# weather_[train/test].csv
# Weather data from a meteorological station as close as possible to the site.

# site_id
# air_temperature - Degrees Celsius
# cloud_coverage - Portion of the sky covered in clouds, in oktas
# dew_temperature - Degrees Celsius
# precip_depth_1_hr - Millimeters
# sea_level_pressure - Millibar/hectopascals
# wind_direction - Compass direction (0-360)
# wind_speed - Meters per second

### Preprocessing/EDA

In [5]:
train=train.merge(building, on=['building_id'], how='left')
train=train.merge(weather_train, on=['timestamp', 'site_id'], how='left')

In [6]:
test=test.merge(building, on=['building_id'], how='left')
test=test.merge(weather_test, on=['timestamp', 'site_id'], how='left')

In [7]:
del weather_train, weather_test, building
gc.collect()

463

In [8]:
#get_stats(train)

In [9]:
#get_stats(test)

In [10]:
drop_cols = ['precip_depth_1_hr'
             , 'year_built'
             , 'floor_count'
             , 'cloud_coverage'
             , 'sea_level_pressure'
             #, 'wind_direction'
             , 'site_id'
            ]
train.drop(drop_cols, axis=1, inplace=True)
train.head()

,building_id,meter,timestamp,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,Education,7432,25.0,20.0,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,Education,2720,25.0,20.0,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,Education,5376,25.0,20.0,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,Education,23685,25.0,20.0,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,Education,116607,25.0,20.0,0.0,0.0


In [11]:
test.drop(drop_cols, axis=1, inplace=True)
test.head()

,row_id,building_id,meter,timestamp,primary_use,square_feet,air_temperature,dew_temperature,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,Education,7432,17.8,11.7,100.0,3.6
1,1,1,0,2017-01-01 00:00:00,Education,2720,17.8,11.7,100.0,3.6
2,2,2,0,2017-01-01 00:00:00,Education,5376,17.8,11.7,100.0,3.6
3,3,3,0,2017-01-01 00:00:00,Education,23685,17.8,11.7,100.0,3.6
4,4,4,0,2017-01-01 00:00:00,Education,116607,17.8,11.7,100.0,3.6


In [12]:
#get_stats(test)

In [13]:
#get_stats(train)

In [14]:
#Drop rows with NA
train.dropna(inplace=True)

#fillna for test
test['air_temperature'].fillna(test['air_temperature'].median(), inplace=True)
test['dew_temperature'].fillna(test['dew_temperature'].median(), inplace=True)
test['wind_speed'].fillna(test['wind_speed'].median(), inplace=True)
test['wind_direction'].fillna(test['wind_direction'].median(), inplace=True)

In [15]:
#encode 'primary_use'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder().fit(train['primary_use'])
train['primary_use'] = le.transform(train['primary_use'])
test['primary_use'] = le.transform(test['primary_use'])

# One-hot?

In [16]:
#fit data type according to the data
train['primary_use'] = train['primary_use'].astype('int8')
train['air_temperature'] = train['air_temperature'].astype('int8')
train['dew_temperature'] = train['dew_temperature'].astype('int8')
train['wind_speed'] = train['wind_speed'].astype('float32')
train['wind_direction'] = train['wind_direction'].astype('int16')
train['building_id'] = train['building_id'].astype('int16')
train['meter'] = train['meter'].astype('int8')
train['meter_reading'] = train['meter_reading'].astype('int32')
train['square_feet'] = train['square_feet'].astype('int32')

test['primary_use'] = test['primary_use'].astype('int8')
test['air_temperature'] = test['air_temperature'].astype('int8')
test['dew_temperature'] = test['dew_temperature'].astype('int8')
test['wind_speed'] = test['wind_speed'].astype('float32')
test['wind_direction'] = test['wind_direction'].astype('int16')
test['building_id'] = test['building_id'].astype('int16')
test['meter'] = test['meter'].astype('int8')
test['square_feet'] = test['square_feet'].astype('int32')

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18761878 entries, 0 to 20216099
Data columns (total 10 columns):
building_id        int16
meter              int8
timestamp          object
meter_reading      int32
primary_use        int8
square_feet        int32
air_temperature    int8
dew_temperature    int8
wind_direction     int16
wind_speed         float32
dtypes: float32(1), int16(2), int32(2), int8(4), object(1)
memory usage: 644.1+ MB


In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 10 columns):
row_id             int64
building_id        int16
meter              int8
timestamp          object
primary_use        int8
square_feet        int32
air_temperature    int8
dew_temperature    int8
wind_direction     int16
wind_speed         float32
dtypes: float32(1), int16(2), int32(1), int64(1), int8(4), object(1)
memory usage: 1.6+ GB


In [19]:

# building_id=175
# meter_id=1

# some_building=train.query(f'building_id=={building_id} and meter=={meter_id}')[['meter_reading', 'timestamp']]
# some_building['timestamp']=pd.to_datetime(some_building['timestamp'])
# some_building.set_index('timestamp', inplace=True)
# some_building.plot() 
# #some_building['2016-08-01':'2016-08-03'].plot() 

#### Feature engineering

In [20]:
# # impute temperature and windspeed by mean value
# train['air_temperature'].fillna(train['air_temperature'].median(), inplace=True)
# train['dew_temperature'].fillna(train['dew_temperature'].median(), inplace=True)
# train['wind_speed'].fillna(train['wind_speed'].median(), inplace=True)

# test['air_temperature'].fillna(test['air_temperature'].median(), inplace=True)
# test['dew_temperature'].fillna(test['dew_temperature'].median(), inplace=True)
# test['wind_speed'].fillna(test['wind_speed'].median(), inplace=True)

In [21]:
#parse timestamp
train['timestamp'] = pd.to_datetime(train['timestamp'])
train['hour'] = train['timestamp'].dt.hour.astype('int8')
train['day'] = train['timestamp'].dt.day.astype('int8')
train['weekday'] = train['timestamp'].dt.weekday.astype('int8')
train['month'] = train['timestamp'].dt.month.astype('int8')
train['weekend'] = np.where(train['weekday']>=5, 1, 0).astype('int8')
train['work_time'] = np.where((train['hour']>=9)&(train['hour']<=18)&(train['weekend']==0), 1, 0).astype('int8')
train['season'] = ((train['month']%12 + 3)//3).astype('int8')
train.drop('timestamp', axis=1, inplace=True)
train.head()

,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_direction,wind_speed,hour,day,weekday,month,weekend,work_time,season
0,0,0,0,0,7432,25,20,0,0.0,0,1,4,1,0,0,1
1,1,0,0,0,2720,25,20,0,0.0,0,1,4,1,0,0,1
2,2,0,0,0,5376,25,20,0,0.0,0,1,4,1,0,0,1
3,3,0,0,0,23685,25,20,0,0.0,0,1,4,1,0,0,1
4,4,0,0,0,116607,25,20,0,0.0,0,1,4,1,0,0,1


In [22]:
#parse timestamp
test['timestamp'] = pd.to_datetime(test['timestamp'])
test['hour'] = test['timestamp'].dt.hour.astype('int8')
test['day'] = test['timestamp'].dt.day.astype('int8')
test['weekday'] = test['timestamp'].dt.weekday.astype('int8')
test['month'] = test['timestamp'].dt.month.astype('int8')
test['weekend'] = np.where(test['weekday']>=5, 1, 0)
test['work_time'] = np.where((test['hour']>=9)&(test['hour']<=18)&(test['weekend']==0), 1, 0).astype('int8')
test['season'] = ((test['month']%12 + 3)//3).astype('int8')
test.drop('timestamp', axis=1, inplace=True)
test.head()

,row_id,building_id,meter,primary_use,square_feet,air_temperature,dew_temperature,wind_direction,wind_speed,hour,day,weekday,month,weekend,work_time,season
0,0,0,0,0,7432,17,11,100,3.6,0,1,6,1,1,0,1
1,1,1,0,0,2720,17,11,100,3.6,0,1,6,1,1,0,1
2,2,2,0,0,5376,17,11,100,3.6,0,1,6,1,1,0,1
3,3,3,0,0,23685,17,11,100,3.6,0,1,6,1,1,0,1
4,4,4,0,0,116607,17,11,100,3.6,0,1,6,1,1,0,1


In [23]:
%%script false
train['consumption'] = train.groupby(['meter', 'air_temperature', 'primary_use', 'hour'])['meter_reading'].transform('sum')
train['square'] = train.groupby(['meter', 'air_temperature', 'primary_use', 'hour'])['square_feet'].transform('sum')
train['consumption_square_ratio'] = (train['consumption']/train['square'])
train.drop(['consumption','square'],axis=1, inplace=True)

Couldn't find program: 'false'


In [24]:
%%script false
map_dict=train.groupby(['meter', 'air_temperature', 'primary_use', 'hour'])['consumption_square_ratio'].median().to_dict()

Couldn't find program: 'false'


In [25]:
%%script false
test=test.set_index(['meter', 'air_temperature', 'primary_use', 'hour'])
test['consumption_square_ratio']=test.index.map(map_dict)
test.reset_index(inplace=True)
test.head()

Couldn't find program: 'false'


In [26]:
test = test.reindex(columns=train.columns)
test.head()

,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_direction,wind_speed,hour,day,weekday,month,weekend,work_time,season
0,0,0,NaN,0,7432,17,11,100,3.6,0,1,6,1,1,0,1
1,1,0,NaN,0,2720,17,11,100,3.6,0,1,6,1,1,0,1
2,2,0,NaN,0,5376,17,11,100,3.6,0,1,6,1,1,0,1
3,3,0,NaN,0,23685,17,11,100,3.6,0,1,6,1,1,0,1
4,4,0,NaN,0,116607,17,11,100,3.6,0,1,6,1,1,0,1


In [27]:
# #looks like there may be some errors with some of the readings
# train[train['site_id'] == 0].plot('timestamp', 'meter_reading')

### Modelling

In [28]:
X_train = train.drop('meter_reading', axis=1)
X_test = test.drop('meter_reading', axis=1)
y = np.log1p(train[['meter_reading']].values.reshape(-1,1)).squeeze()

del train
del test
gc.collect()


762

In [29]:
from sklearn.model_selection import KFold
import lightgbm as lgb
gc.collect()
kf = KFold(n_splits = 3, shuffle = True, random_state=42)
y_valid_pred = np.zeros(X_train.shape[0])
models = []

for tr_idx, val_idx in kf.split(X_train, y):
    tr_x, tr_y = X_train.iloc[tr_idx,:], y[tr_idx]
    vl_x, vl_y = X_train.iloc[val_idx,:], y[val_idx]
            
    print(len(tr_x),len(vl_x)) 
    
    clf = lgb.LGBMRegressor(n_estimators=100,learning_rate=0.2,feature_fraction=0.9,
            bagging_fraction=0.9,early_stopping_rounds=50)
    clf.fit(tr_x, tr_y,
        eval_set=[(vl_x, vl_y)],
        verbose=2)
    y_valid_pred[val_idx] += clf.predict(vl_x, num_iteration=clf.best_iteration_)
    models.append(clf)
    gc.collect()
    
#predict test  
y_test_pred = np.zeros(X_test.shape[0])
for clf in models:
      y_test_pred += np.expm1(clf.predict(X_test, num_iteration=clf.best_iteration_))
y_test_pred /= len(models)

12507918 6253960


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 4.19124
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l2: 3.79895
[3]	valid_0's l2: 3.49963
[4]	valid_0's l2: 3.28003
[5]	valid_0's l2: 3.08992
[6]	valid_0's l2: 2.95412
[7]	valid_0's l2: 2.85419
[8]	valid_0's l2: 2.77207
[9]	valid_0's l2: 2.68397
[10]	valid_0's l2: 2.61015
[11]	valid_0's l2: 2.54238
[12]	valid_0's l2: 2.48462
[13]	valid_0's l2: 2.43882
[14]	valid_0's l2: 2.37637
[15]	valid_0's l2: 2.3359
[16]	valid_0's l2: 2.29739
[17]	valid_0's l2: 2.2565
[18]	valid_0's l2: 2.20755
[19]	valid_0's l2: 2.18282
[20]	valid_0's l2: 2.16677
[21]	valid_0's l2: 2.13947
[22]	valid_0's l2: 2.12447
[23]	valid_0's l2: 2.09734
[24]	valid_0's l2: 2.06862
[25]	valid_0's l2: 2.04727
[26]	valid_0's l2: 2.02985
[27]	valid_0's l2: 2.01652
[28]	valid_0's l2: 2.00601
[29]	valid_0's l2: 1.99142
[30]	valid_0's l2: 1.97636
[31]	valid_0's l2: 1.95853
[32]	valid_0's l2: 1.94732
[33]	valid_0's l2: 1.93041
[34]	valid_0's l2: 1.92054
[35]	valid_0's l2: 1.91232
[36]	

MemoryError: 

Evaluation

In [ ]:
#Since the minimum value is 0, it was clipped to do so.
print('valid score is',np.sqrt(sum(np.power(y-np.clip(y_valid_pred,0,None),2))/y.shape[0]))

In [ ]:
pd.DataFrame(np.clip(y_test_pred,0,None)).shape

In [ ]:
sub.shape

In [ ]:
sub = pd.read_csv("../data/sample_submission.csv")
sub["meter_reading"] = y_test_pred
sub.head()

In [ ]:
sub.to_csv("submission.csv", index = False)
